In [1]:
# Shiwei SU
# Takai Lab, Department of Bioengineering, School of Engineering
# The University of Tokyo, Japan

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MaxAbsScaler

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold

from statistics import mean, stdev

import shap
%matplotlib inline
shap.initjs()

Dataset

In [ ]:
dataset = pd.read_csv(input("Enter the name of the dataset file: ")+".csv")

columns_to_drop = ["ref", "Sample", "Smiles"]
for column in columns_to_drop:
    if column in dataset:
        dataset = dataset.drop(column, axis=1)

# MaxAbsScaler for descriptors
CA = dataset.filter(regex="Contact")
df_raw = dataset.drop("Contact angle (deg)", axis = 1)
transformer = MaxAbsScaler()
df_scaled = transformer.fit_transform(df_raw)
dataset_scaled = pd.DataFrame(df_scaled, columns=df_raw.columns)
dataset = pd.concat([CA, dataset_scaled], axis = 1)

# Distribution
y_name = "Contact Angle"
number_of_bins = 7
if dataset.iloc[:, 0].dtype=="float":
    plt.figure(figsize=(7.5, 3))
    plt.rcParams["font.size"] = 18
    plt.hist(dataset.iloc[:, 0], bins=number_of_bins, color="dodgerblue")
    plt.xlabel("Contact Angle", font="Arial")
    plt.ylabel("Frequency", font="Arial")
    plt.xticks([0, 30, 60, 90, 120])
    plt.yticks([0,5,10,15,20,25,30])

Hyperparameter

In [6]:
# Hyperparameter Tuning
param_lasso = {"alpha": np.arange(0.05, 3., 0.01),
              "fit_intercept": [True, False]}

param_ridge = {"alpha": np.arange(0.05, 3., 0.01),
              "fit_intercept": [True, False]}

param_tree = {"max_depth": np.arange(1, 20),
             "min_samples_leaf": np.arange(1, 10, 1),
             "random_state": [42]}

param_forest = {"max_depth": np.arange(1, 20),
             "min_samples_leaf": np.arange(1, 10, 1),
             "n_estimators": [50, 100, 200, 300],
             "random_state": [42]}

param_knn = {"n_neighbors": np.arange(1, 10),
              "weights": ["uniform", "distance"]}

param_svr = {"C": [0.1, 1, 10, 100, 1000],
              "epsilon": [0.01, 0.1, 1, 10],
              "kernel": ["linear", "poly", "rbf", "sigmoid"]}

param_mlp = {"hidden_layer_sizes": [(50,), (100,), (150,),
                                    (50, 50), (100, 100), (150, 150),
                                    (50, 100), (50, 150), (100, 150),
                                    (50,50,50), (100,100,100), (150, 150, 150),
                                    (50, 100, 50), (50, 150, 50), (100, 100, 150), (100, 150, 100)],
             "activation": ["relu", "logistic", "tanh"],
             "solver": ["adam", "sgd"],
             "alpha": [0.0001, 0.001, 0.01],
             "learning_rate_init": [0.0001, 0.001, 0.01],
             "shuffle": [False],
             "max_iter": [1000]}

param_xgb = {"max_depth": np.arange(1, 20),
             "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],
             "n_estimators": [50, 100, 150, 250, 500, 1000],
             "random_state": [42]}

models_param = {
    "Lasso": {"model": Lasso(), "param": param_lasso},
    "Ridge": {"model": Ridge(), "param": param_ridge},
    "Decision Tree": {"model": DecisionTreeRegressor(), "param": param_tree},
    "Random Forest": {"model": RandomForestRegressor(), "param": param_forest},
    "kNN": {"model": KNeighborsRegressor(), "param": param_knn},
    "SVR": {"model": SVR(), "param": param_svr},
    "MLP": {"model": MLPRegressor(), "param": param_mlp},
    "XGBoost": {"model": XGBRegressor(), "param": param_xgb}}

optimized_models = {"Lasso":{},
                    "Ridge":{},
                    "Decision Tree":{},
                    "Random Forest":{},
                    "kNN":{},
                    "SVR":{},
                    "MLP":{},
                    "XGBoost":{}



Nested CV

In [ ]:
X = np.array(dataset.drop("Contact angle (deg)", axis = 1))
Y = np.array(dataset["Contact angle (deg)"])

outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)

for model_name, model_param in models_param.items():
    print("Training "+model_name+"...")
    model = model_param["model"]
    param = model_param["param"]
    best_para = []
    train_scores = []
    test_scores = []

    for train_idx, test_idx in outer_cv.split(X):
        X_train, y_train = X[train_idx], Y[train_idx]
        X_test, y_test = X[test_idx], Y[test_idx]
        grid = GridSearchCV(model, param, cv=inner_cv, n_jobs=-1, verbose=1)
        grid.fit(X_train, y_train)
        best_para.append(grid.best_params_)
        train_scores.append(grid.best_score_)
        model = grid.best_estimator_.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2_pred = r2_score(y_test, y_pred)
        test_scores.append(r2_pred)
    optimized_models[model_name]["best_para"] = best_para
    optimized_models[model_name]["train_scores"] = train_scores
    optimized_models[model_name]["test_scores"] = test_scores


    print("-------------------------NEXT-------------------------")

In [10]:
#split validation set
train_set, validation_set = train_test_split(dataset, test_size=0.2, random_state=42)

#training set
X_part = train_set.drop("Contact angle (deg)", axis=1)
Y_part = train_set["Contact angle (deg)"].copy()
X_part_array = np.array(X_part)
Y_part_array = np.array(Y_part)

#validation set
X_val = validation_set.drop("Contact angle (deg)", axis=1)
Y_val = validation_set["Contact angle (deg)"].copy()
X_val_array = np.array(X_val)
Y_val_array = np.array(Y_val)

In [11]:
def gen_fig(model_name, X_train, Y_train, X_val, Y_val, rmse_train, rmse_val, r2_train, r2_val, std_part_predictions, std_predictions):

    model_name = str(model_name)
    fig,ax=plt.subplots(figsize=(6, 6))
    Axis_line=np.linspace(*ax.get_xlim(),2)
    ax.plot(Axis_line,Axis_line,transform=ax.transAxes,linestyle="--",
            linewidth=2,color='black',label="1:1 Line")
    ax.set_xlim([0,195])
    ax.set_ylim([0,195])
    
    rmse_train_mean = round(np.mean(rmse_train, axis=0), 2)
    rmse_val_mean = round(np.mean(rmse_val, axis=0), 2)
    rmse_train_std = round(np.std(rmse_train, axis=0), 2)
    rmse_val_std = round(np.std(rmse_val, axis=0), 2)
    r2_train_mean = round(np.mean(r2_train, axis=0), 2)
    r2_val_mean = round(np.mean(r2_val, axis=0), 2)
    r2_train_std = round(np.std(r2_train, axis=0), 2)
    r2_val_std = round(np.std(r2_val, axis=0), 2)
    
    scatter1 = ax.scatter(X_train, Y_train, color="salmon", alpha=0.5, label="Training set")
    plt1 = ax.errorbar(X_train, Y_train, yerr=std_part_predictions, fmt="none", color="r", alpha=0.5)
    scatter2 = ax.scatter(X_val, Y_val, color="steelblue", label="Test set")
    plt2 = ax.errorbar(X_val, Y_val, yerr=std_predictions, fmt="none", color="b", alpha=0.5)
    plt.legend(handles=[scatter1, scatter2], labels = ["Training set", "Test set"], loc="lower right", fontsize=14, fancybox=True, markerscale=0.8, framealpha=0.8, handlelength=0.5)

    ax.text(5, 190, " Score of "+model_name+" "
            "\n $RMSE$ (training): "+str(rmse_train_mean)+" ± "+str(rmse_train_std)+" "
            "\n $RMSE$ (test): "+str(rmse_val_mean)+" ± "+str(rmse_val_std)+" "
            "\n $R\u00b2$ (training): "+str(r2_train_mean)+" ± "+str(r2_train_std)+" "
            "\n $R\u00b2$ (test): "+str(r2_val_mean)+" ± "+str(r2_val_std)+" ",
            verticalalignment='top', horizontalalignment='left',fontsize=16,
            bbox={'facecolor': 'lightskyblue', 'alpha': 0.2, 'pad': 1})

    plt.xlabel("Reported CA ", fontsize=20, fontweight="bold", font="Arial")
    plt.ylabel("Predicted CA", fontsize=20, fontweight="bold", font="Arial")
    new_ticks = np.linspace(0, 180, 7)
    plt.xticks(new_ticks, fontweight="bold")
    plt.yticks(new_ticks, fontweight="bold")
    ax.tick_params(labelsize="16", direction="in")

    plt.rcParams["axes.titley"] = 1.03
    plt.title(model_name, fontsize=26,fontweight="bold",font="Arial", color="black")
            
    plt.show()
    plt.clf()

In [ ]:
def result_lasso(optimized_models, model_name, X_part_array, Y_part_array, X_val_array, Y_val_array):
    all_y_predictions = []
    all_y_part_predictions = []
    rmse_train = []
    rmse_val = []
    r2_train = []
    r2_val = []
    for i in range(5):
        model = Lasso(**optimized_models["Lasso"]["best_para"][i])
        model.fit(X_part_array, Y_part_array)
        y_part_pred = model.predict(X_part_array)
        y_pred = model.predict(X_val_array)
        all_y_part_predictions.append(y_part_pred)
        all_y_predictions.append(y_pred)
        rmse_train.append(np.sqrt(mean_squared_error(Y_part_array, y_part_pred)))
        rmse_val.append(np.sqrt(mean_squared_error(Y_val_array, y_pred)))
        r2_train.append(r2_score(Y_part_array, y_part_pred))
        r2_val.append(r2_score(Y_val_array, y_pred))       

    mean_part_predictions = np.mean(all_y_part_predictions, axis=0)
    std_part_predictions = np.std(all_y_part_predictions, axis=0)
    mean_predictions = np.mean(all_y_predictions, axis=0)
    std_predictions = np.std(all_y_predictions, axis=0)

    gen_fig(model_name, Y_part_array, mean_part_predictions, Y_val_array, mean_predictions, rmse_train, rmse_val, r2_train, r2_val, std_part_predictions, std_predictions)
    
result_lasso(optimized_models, "Lasso", X_part_array, Y_part_array, X_val_array, Y_val_array)


In [ ]:
def result_ridge(optimized_models, model_name, X_part_array, Y_part_array, X_val_array, Y_val_array):
    all_y_predictions = []
    all_y_part_predictions = []
    rmse_train = []
    rmse_val = []
    r2_train = []
    r2_val = []
    for i in range(5):
        model = Ridge(**optimized_models["Ridge"]["best_para"][i])
        model.fit(X_part_array, Y_part_array)
        y_part_pred = model.predict(X_part_array)
        y_pred = model.predict(X_val_array)
        all_y_part_predictions.append(y_part_pred)
        all_y_predictions.append(y_pred)
        rmse_train.append(np.sqrt(mean_squared_error(Y_part_array, y_part_pred)))
        rmse_val.append(np.sqrt(mean_squared_error(Y_val_array, y_pred)))
        r2_train.append(r2_score(Y_part_array, y_part_pred))
        r2_val.append(r2_score(Y_val_array, y_pred))     
        
    mean_part_predictions = np.mean(all_y_part_predictions, axis=0)
    std_part_predictions = np.std(all_y_part_predictions, axis=0)
    mean_predictions = np.mean(all_y_predictions, axis=0)
    std_predictions = np.std(all_y_predictions, axis=0)

    gen_fig(model_name, Y_part_array, mean_part_predictions, Y_val_array, mean_predictions, rmse_train, rmse_val, r2_train, r2_val, std_part_predictions, std_predictions)
    
result_ridge(optimized_models, "Ridge", X_part_array, Y_part_array, X_val_array, Y_val_array)

In [ ]:
def result_tree(optimized_models, model_name, X_part_array, Y_part_array, X_val_array, Y_val_array):
    all_y_predictions = []
    all_y_part_predictions = []
    rmse_train = []
    rmse_val = []
    r2_train = []
    r2_val = []
    for i in range(5):
        model = DecisionTreeRegressor(**optimized_models["Decision Tree"]["best_para"][i])
        model.fit(X_part_array, Y_part_array)
        y_part_pred = model.predict(X_part_array)
        y_pred = model.predict(X_val_array)
        all_y_part_predictions.append(y_part_pred)
        all_y_predictions.append(y_pred)
        rmse_train.append(np.sqrt(mean_squared_error(Y_part_array, y_part_pred)))
        rmse_val.append(np.sqrt(mean_squared_error(Y_val_array, y_pred)))
        r2_train.append(r2_score(Y_part_array, y_part_pred))
        r2_val.append(r2_score(Y_val_array, y_pred))   
    mean_part_predictions = np.mean(all_y_part_predictions, axis=0)
    std_part_predictions = np.std(all_y_part_predictions, axis=0)
    mean_predictions = np.mean(all_y_predictions, axis=0)
    std_predictions = np.std(all_y_predictions, axis=0)
    
    gen_fig(model_name, Y_part_array, mean_part_predictions, Y_val_array, mean_predictions, rmse_train, rmse_val, r2_train, r2_val, std_part_predictions, std_predictions)
    
result_tree(optimized_models, "Decision Tree", X_part_array, Y_part_array, X_val_array, Y_val_array)

In [ ]:
def result_forest(optimized_models, model_name, X_part_array, Y_part_array, X_val_array, Y_val_array):
    all_y_predictions = []
    all_y_part_predictions = []
    rmse_train = []
    rmse_val = []
    r2_train = []
    r2_val = []
    for i in range(5):
        model = RandomForestRegressor(**optimized_models["Random Forest"]["best_para"][i])
        model.fit(X_part_array, Y_part_array)
        y_part_pred = model.predict(X_part_array)
        y_pred = model.predict(X_val_array)
        all_y_part_predictions.append(y_part_pred)
        all_y_predictions.append(y_pred)
        rmse_train.append(np.sqrt(mean_squared_error(Y_part_array, y_part_pred)))
        rmse_val.append(np.sqrt(mean_squared_error(Y_val_array, y_pred)))
        r2_train.append(r2_score(Y_part_array, y_part_pred))
        r2_val.append(r2_score(Y_val_array, y_pred))   
    mean_part_predictions = np.mean(all_y_part_predictions, axis=0)
    std_part_predictions = np.std(all_y_part_predictions, axis=0)
    mean_predictions = np.mean(all_y_predictions, axis=0)
    std_predictions = np.std(all_y_predictions, axis=0)
    gen_fig(model_name, Y_part_array, mean_part_predictions, Y_val_array, mean_predictions, rmse_train, rmse_val, r2_train, r2_val, std_part_predictions, std_predictions)
    
result_forest(optimized_models, "Random Forest", X_part_array, Y_part_array, X_val_array, Y_val_array)

In [ ]:
def result_knn(optimized_models, model_name, X_part_array, Y_part_array, X_val_array, Y_val_array):
    all_y_predictions = []
    all_y_part_predictions = []
    rmse_train = []
    rmse_val = []
    r2_train = []
    r2_val = []
    for i in range(5):
        model = KNeighborsRegressor(**optimized_models["kNN"]["best_para"][i])
        model.fit(X_part_array, Y_part_array)
        y_part_pred = model.predict(X_part_array)
        y_pred = model.predict(X_val_array)
        all_y_part_predictions.append(y_part_pred)
        all_y_predictions.append(y_pred)
        rmse_train.append(np.sqrt(mean_squared_error(Y_part_array, y_part_pred)))
        rmse_val.append(np.sqrt(mean_squared_error(Y_val_array, y_pred)))
        r2_train.append(r2_score(Y_part_array, y_part_pred))
        r2_val.append(r2_score(Y_val_array, y_pred))   
    mean_part_predictions = np.mean(all_y_part_predictions, axis=0)
    std_part_predictions = np.std(all_y_part_predictions, axis=0)
    mean_predictions = np.mean(all_y_predictions, axis=0)
    std_predictions = np.std(all_y_predictions, axis=0)
    gen_fig(model_name, Y_part_array, mean_part_predictions, Y_val_array, mean_predictions, rmse_train, rmse_val, r2_train, r2_val, std_part_predictions, std_predictions)

result_knn(optimized_models, "kNN", X_part_array, Y_part_array, X_val_array, Y_val_array)

In [ ]:
def result_svr(optimized_models, model_name, X_part_array, Y_part_array, X_val_array, Y_val_array):
    all_y_predictions = []
    all_y_part_predictions = []
    rmse_train = []
    rmse_val = []
    r2_train = []
    r2_val = []
    for i in range(5):
        model = SVR(**optimized_models["SVR"]["best_para"][i])
        model.fit(X_part_array, Y_part_array)
        y_part_pred = model.predict(X_part_array)
        y_pred = model.predict(X_val_array)
        all_y_part_predictions.append(y_part_pred)
        all_y_predictions.append(y_pred)
        rmse_train.append(np.sqrt(mean_squared_error(Y_part_array, y_part_pred)))
        rmse_val.append(np.sqrt(mean_squared_error(Y_val_array, y_pred)))
        r2_train.append(r2_score(Y_part_array, y_part_pred))
        r2_val.append(r2_score(Y_val_array, y_pred))   
    mean_part_predictions = np.mean(all_y_part_predictions, axis=0)
    std_part_predictions = np.std(all_y_part_predictions, axis=0)
    mean_predictions = np.mean(all_y_predictions, axis=0)
    std_predictions = np.std(all_y_predictions, axis=0)
    gen_fig(model_name, Y_part_array, mean_part_predictions, Y_val_array, mean_predictions, rmse_train, rmse_val, r2_train, r2_val, std_part_predictions, std_predictions)
    return model
    
svr = result_svr(optimized_models, "SVR", X_part_array, Y_part_array, X_val_array, Y_val_array)

In [ ]:
def result_mlp(optimized_models, model_name, X_part_array, Y_part_array, X_val_array, Y_val_array):
    all_y_predictions = []
    all_y_part_predictions = []
    rmse_train = []
    rmse_val = []
    r2_train = []
    r2_val = []
    for i in range(5):
        model = MLPRegressor(**optimized_models["MLP"]["best_para"][i])
        model.fit(X_part_array, Y_part_array)
        y_part_pred = model.predict(X_part_array)
        y_pred = model.predict(X_val_array)
        all_y_part_predictions.append(y_part_pred)
        all_y_predictions.append(y_pred)
        rmse_train.append(np.sqrt(mean_squared_error(Y_part_array, y_part_pred)))
        rmse_val.append(np.sqrt(mean_squared_error(Y_val_array, y_pred)))
        r2_train.append(r2_score(Y_part_array, y_part_pred))
        r2_val.append(r2_score(Y_val_array, y_pred))   
    mean_part_predictions = np.mean(all_y_part_predictions, axis=0)
    std_part_predictions = np.std(all_y_part_predictions, axis=0)
    mean_predictions = np.mean(all_y_predictions, axis=0)
    std_predictions = np.std(all_y_predictions, axis=0)
    gen_fig(model_name, Y_part_array, mean_part_predictions, Y_val_array, mean_predictions, rmse_train, rmse_val, r2_train, r2_val, std_part_predictions, std_predictions)
    
result_mlp(optimized_models, "MLP", X_part_array, Y_part_array, X_val_array, Y_val_array)

In [ ]:
def result_xgb(optimized_models, model_name, X_part_array, Y_part_array, X_val_array, Y_val_array):
    all_y_predictions = []
    all_y_part_predictions = []
    rmse_train = []
    rmse_val = []
    r2_train = []
    r2_val = []
    for i in range(5):
        model = XGBRegressor(**optimized_models["XGBoost"]["best_para"][i])
        model.fit(X_part_array, Y_part_array)
        y_part_pred = model.predict(X_part_array)
        y_pred = model.predict(X_val_array)
        all_y_part_predictions.append(y_part_pred)
        all_y_predictions.append(y_pred)
        rmse_train.append(np.sqrt(mean_squared_error(Y_part_array, y_part_pred)))
        rmse_val.append(np.sqrt(mean_squared_error(Y_val_array, y_pred)))
        r2_train.append(r2_score(Y_part_array, y_part_pred))
        r2_val.append(r2_score(Y_val_array, y_pred))   
    mean_part_predictions = np.mean(all_y_part_predictions, axis=0)
    std_part_predictions = np.std(all_y_part_predictions, axis=0)
    mean_predictions = np.mean(all_y_predictions, axis=0)
    std_predictions = np.std(all_y_predictions, axis=0)
    gen_fig(model_name, Y_part_array, mean_part_predictions, Y_val_array, mean_predictions, rmse_train, rmse_val, r2_train, r2_val, std_part_predictions, std_predictions)
    return model
    
xgb_model = result_xgb(optimized_models, "XGBoost", X_part_array, Y_part_array, X_val_array, Y_val_array)

Interpretability

SVR

In [ ]:
def shap_svr(optimized_models, model_name, X_part_array, Y_part_array, X_val_array, Y_val_array):
    for i in range(5):
        model = SVR(**optimized_models["SVR"]["best_para"][i])
        model.fit(X_part_array, Y_part_array)
        explainer = shap.KernelExplainer(model.predict, X_part_array)
        shap_values = explainer.shap_values(X_val_array)
        shap.summary_plot(shap_values, X_val_array, feature_names=X_part.columns.values, plot_size=(8,5), color_bar_label="Feature SHAP Value", auto_size_plot=None)
        plt.show()
    
shap_svr(optimized_models, "SVR", X_part_array, Y_part_array, X_val_array, Y_val_array)

Random Forest

In [ ]:
def shap_forest(optimized_models, X_part_array, Y_part_array, X_val_array):
    for i in range(5):
        model = RandomForestRegressor(**optimized_models["Random Forest"]["best_para"][i])
        model.fit(X_part_array, Y_part_array)
        explainer = shap.KernelExplainer(model.predict, X_part_array)
        shap_values = explainer.shap_values(X_val_array)
        shap.summary_plot(shap_values, X_val_array, feature_names=X_part.columns.values, plot_size=(8,5), color_bar_label="Feature SHAP Value", auto_size_plot=None)
        plt.show()
    
shap_forest(optimized_models, X_part_array, Y_part_array, X_val_array)

XGBoost

In [ ]:
def shap_xgb(optimized_models, X_part, Y_part, X_val):
    for i in range(5):
        model = XGBRegressor(**optimized_models["XGBoost"]["best_para"][i])
        model.fit(X_part, Y_part)
        explainer = shap.KernelExplainer(model.predict, X_part)
        shap_values = explainer.shap_values(X_val)

        shap.summary_plot(shap_values, X_val, feature_names=X_part.columns.values, plot_size=(8,5), color_bar_label="Feature SHAP Value", auto_size_plot=None)
        plt.show()
        
shap_xgb(optimized_models, X_part, Y_part, X_val)